In [57]:
#importing libraries
#reference : https://arxiv.org/pdf/1409.3215.pdf


import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
tf.reset_default_graph()

In [58]:
#some hyper parameters

encoder_vocab_size = 100  # suppose our encoding data vocab size is 100 
decoder_vocab_size = 150  # and decoding data vocab size is 150

batch_size=10

encoder_hidden_unit = 100
decoder_hidden_unit = 100

encoder_embedding_dim=50
decoder_embedding_dim=50


In [59]:
#inputs :  encoder_input , decoder_input , decoder_target

encoder_input = tf.placeholder(name='encoder_input',shape=[None,None],dtype=tf.int32)
decoder_input = tf.placeholder(name='decoder_input',shape=[None,None],dtype=tf.int32)
decoder_target= tf.placeholder(name='decoder_target',shape=[None,None],dtype=tf.int32)

#ecoder input should be  [Max_time,batch]  time major  
#decoder input should be [Max_time,batch]  time major
#decoder_target should be [Max_time,batch] time major
print(encoder_input)

Tensor("encoder_input:0", shape=(?, ?), dtype=int32)


In [60]:
#embedding for encoder , decoder


#use tf.get_variable instead of tf.Variable
encoder_embedding = tf.get_variable(name='encoder_embedding',
                                    shape=[encoder_vocab_size,encoder_embedding_dim],
                                    dtype=tf.float32,
                                    initializer=tf.random_uniform_initializer(-0.01,0.01))

decoder_embedding = tf.get_variable(name='decoder_embedding',
                                    shape=[decoder_vocab_size,decoder_embedding_dim],
                                    dtype=tf.float32,
                                    initializer=tf.random_uniform_initializer(-0.01,0.01))

embedd_encoder = tf.nn.embedding_lookup(encoder_embedding,encoder_input)
#now encoder input will become [ max_time , batch_size , embedding_dim ]

embedd_decoder = tf.nn.embedding_lookup(decoder_embedding,decoder_input)
#now decoder input will become [ max_time , batch_size , embedding_dim ]

print(embedd_encoder)

Tensor("embedding_lookup:0", shape=(?, ?, 50), dtype=float32)


In [61]:
#cell for rnn

encoder_cell = rnn.LSTMCell(num_units=encoder_hidden_unit)
decoder_cell = rnn.LSTMCell(num_units=decoder_hidden_unit)

In [62]:
#encoder

encoder_model, encoder_last_state = tf.nn.dynamic_rnn(cell=encoder_cell,
                                                      inputs=embedd_encoder,
                                                      time_major=True,
                                                      dtype=tf.float32)

In [63]:
#decoder 
decoder_output,decoder_last_state= tf.nn.dynamic_rnn(cell=decoder_cell,
                                                     inputs=embedd_decoder,
                                                     time_major=True,
                                                     initial_state=encoder_last_state,
                                                     dtype=tf.float32,
                                                     scope='decoder_inputs')


In [64]:
#projection layer without activation

linear_projection = tf.contrib.layers.fully_connected(decoder_output,decoder_vocab_size)


#taking max argument
prediction = tf.argmax(linear_projection,2)


In [65]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(decoder_target,
                                                     depth=decoder_vocab_size,dtype=tf.float32),
                                                     logits=linear_projection)

#reduce_mean
loss=tf.reduce_mean(cross_entropy)

#our aim is to minimize this loss


In [13]:
train = tf.train.AdamOptimizer().minimize(loss)

In [68]:
#let's check network

#make sure your encoder and decoder input and decoder taget batch size is same , time_stamps can be differ but batch_size
#should be same otherwise you will get dim mismatch Error

#fake data
encoder_input_ = np.random.randint(0,10,[5,3])  #time major  [max_time , batch]
print("encoder_input \n",encoder_input_)

decoder_input_ = np.random.randint(0,10,[4,3])  #time_major  [max_time , batch]
print("decoder_input \n",decoder_input_)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    prediction_data = sess.run(prediction,feed_dict={encoder_input:encoder_input_,decoder_input:decoder_input_})
    print("prediction_data \n",prediction_data)

encoder_input 
 [[6 5 9]
 [9 6 8]
 [9 8 7]
 [5 5 7]
 [3 1 5]]
decoder_input 
 [[9 2 3]
 [1 7 2]
 [5 8 3]
 [7 4 8]]
prediction_data 
 [[ 80 137 117]
 [ 80  74   4]
 [  3  65   4]
 [103  10  51]]
